<a href="https://colab.research.google.com/github/chldnjstjr/Weather-Consumer/blob/main/R_%ED%8C%A8%EB%84%90%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colab에서 R 사용 가능

[링크 텍스트](https://www.youtube.com/watch?v=ugNlbyOlWlQ) #이 분 없었으면 평생 헤맸을듯

In [1]:
system("gdown --id 1q0JuMeYNscfq2BrGuEfk-_fDZTj2y5k-")
library(readxl)

In [2]:
df_master = read_excel("df_master.xlsx") #구글 드라이브에서 주소 가져올 때 링크가 있는 모든 사용자에게 공개 꼭! 해줘야함 

In [3]:
head(df_master)

CLNT_ID,SESS_ID,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,PD_NM,CLAC1_NM,CLAC2_NM,⋯,TEMP_AVG,RAINFALL_HR,RAINFALL_DAY,WIND_AVG,DEWPT_AVG,HUMID_AVG,STEAMP_AVG,ATMOS_AGV,SOLAR_SUM,DUST
<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4316011,2889247,574397,1개,농협홍삼한삼인,39000,1,한삼인 홍삼정 굿데이스틱 60포/쇼핑백증정,건강식품,홍삼/인삼가공식품,⋯,30.5,1.58,1.0,1.4,23.7,68.6,29.3,1000.4,12.29,33
4351904,8220376,747273,(not set),[새남해농협],39000,1,새남해농협 해두룸 흑마늘진액 프리미엄 70ml 20팩,건강식품,건강진액,⋯,23.2,8.13,28.3,1.9,22.3,95.4,27.0,1001.4,1.50,38
4351904,8220376,803015,(not set),[새남해농협],45430,1,[새남해농협]해두룸 흑마늘진액(80ml*30봉),건강식품,건강진액,⋯,23.2,8.13,28.3,1.9,22.3,95.4,27.0,1001.4,1.50,38
4395026,7473082,722396,1개,천지인,39000,1,[동원천지인] 홍삼정 데일리원 (10mlx30포),건강식품,홍삼/인삼가공식품,⋯,22.3,0.00,0.0,1.6,11.0,49.8,13.2,999.9,26.49,44
4549371,4421328,563452,1개,정관장.,61500,1,◆ 홍삼원기 (50ml*30포)X2박스 + 쇼핑백(2개)증정,건강식품,홍삼/인삼가공식품,⋯,26.1,0.00,0.0,2.2,22.2,81.0,26.8,1004.0,27.78,19
4549371,4421328,372235,1개,천호엔케어,94000,1,[#텐텐] 온기 담은 3년강화사자발쑥 100ml 30개입 2박스,건강식품,건강진액,⋯,26.1,0.00,0.0,2.2,22.2,81.0,26.8,1004.0,27.78,19


In [11]:
unique(df_master$CLAC1_NM)

[1] "건강식품" "냉동식품" "냉장식품" "과일"     "축산물"

In [12]:
df_master_health <- subset(df_master, CLAC1_NM=="건강식품") 
df_master_ice <- subset(df_master, CLAC1_NM=="냉동식품") 
df_master_ref <- subset(df_master, CLAC1_NM=="냉장식품") 
df_master_fruit <- subset(df_master, CLAC1_NM=="과일")
df_master_meet  <- subset(df_master, CLAC1_NM=="축산물")

In [6]:
install.packages('plm')
library(plm)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘miscTools’, ‘rbibutils’, ‘bdsmatrix’, ‘zoo’, ‘sandwich’, ‘lmtest’, ‘maxLik’, ‘Rdpack’, ‘Formula’




In [33]:
a=plm(log(PD_AMOUNT) ~ TEMP_AVG+RAINFALL_DAY+HUMID_AVG+DUST,index="CLNT_ID" ,data=df_master_health, model="within")
summary(a)

#출처 https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=vnf3751&logNo=220868710769

Oneway (individual) effect Within Model

Call:
plm(formula = log(PD_AMOUNT) ~ TEMP_AVG + RAINFALL_DAY + HUMID_AVG + 
    DUST, data = df_master_health, model = "within", index = "CLNT_ID")

Unbalanced Panel: n = 66256, T = 1-576, N = 123623

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-5.693951 -0.016951  0.000000  0.014885  3.880780 

Coefficients:
                Estimate  Std. Error t-value  Pr(>|t|)    
TEMP_AVG     -0.00280409  0.00058696 -4.7773 1.781e-06 ***
RAINFALL_DAY  0.00013338  0.00020983  0.6357   0.52498    
HUMID_AVG     0.00060594  0.00025158  2.4085   0.01602 *  
DUST          0.00049406  0.00021586  2.2888   0.02210 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    15296
Residual Sum of Squares: 15285
R-Squared:      0.00071738
Adj. R-Squared: -1.1535
F-statistic: 10.2952 on 4 and 57363 DF, p-value: 2.4824e-08

### 모델 자체가 걍 엉망진창